# Ecriture du modèle à résoudre


In [1]:
from docplex.mp.model import Model


In [2]:
m = Model(name='Découpage éléctoral')

# Variables

In [3]:
#On entre ensuite les constantes du problème
# Pour le cas où on les entre à la main 
pVi = [0.4, 0.6, 0.7, 0.2, 0.9]
nbVi = [5, 6, 5, 6, 7]

n = len(nbVi)
k = 2

In [4]:
#On insère ici toutes les variables nécessaires : les zj, xij et yij

x = m.binary_var_matrix(range(1,n+1),range(1,k+1), name="x", key_format=None)

In [5]:
y = m.binary_var_matrix(range(1,n+1), range(1,k+1), name="y", key_format=None)


In [20]:
z = m.binary_var_list(range(1,k+1), name="z", key_format=None)


In [21]:
print(x)

{(1, 1): docplex.mp.Var(type=B,name='x_1_1'), (1, 2): docplex.mp.Var(type=B,name='x_1_2'), (2, 1): docplex.mp.Var(type=B,name='x_2_1'), (2, 2): docplex.mp.Var(type=B,name='x_2_2'), (3, 1): docplex.mp.Var(type=B,name='x_3_1'), (3, 2): docplex.mp.Var(type=B,name='x_3_2'), (4, 1): docplex.mp.Var(type=B,name='x_4_1'), (4, 2): docplex.mp.Var(type=B,name='x_4_2'), (5, 1): docplex.mp.Var(type=B,name='x_5_1'), (5, 2): docplex.mp.Var(type=B,name='x_5_2')}


In [22]:
print(z)

[docplex.mp.Var(type=B,name='z_1'), docplex.mp.Var(type=B,name='z_2')]


In [23]:
print(z[0])

z_1


In [24]:
print(sum([nbVi[i-1]*x[i,1] for i in range(1, n+1)]))

5x_1_1+6x_2_1+5x_3_1+6x_4_1+7x_5_1


In [25]:
print(sum(x[1,j] for j in range(1, k+1)))

x_1_1+x_1_2


# Constraints

In [26]:
#Additionner les éléments sur les colonnes
m.add_constraints((sum(x[i,j] for j in range(1, k+1)) == 1 for i in range(1, n+1))) 


[docplex.mp.LinearConstraint[](x_1_1+x_1_2,EQ,1),
 docplex.mp.LinearConstraint[](x_2_1+x_2_2,EQ,1),
 docplex.mp.LinearConstraint[](x_3_1+x_3_2,EQ,1),
 docplex.mp.LinearConstraint[](x_4_1+x_4_2,EQ,1),
 docplex.mp.LinearConstraint[](x_5_1+x_5_2,EQ,1)]

In [27]:
m.print_information()

Model: Découpage éléctoral
 - number of variables: 26
   - binary=26, integer=0, continuous=0
 - number of constraints: 10
   - linear=10
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [28]:
#Additionner les éléments sur les lignes
m.add_constraints((sum(x[i,j] for i in range(1, n+1)) >= 1 for j in range(1, k+1))) 


[docplex.mp.LinearConstraint[](x_1_1+x_2_1+x_3_1+x_4_1+x_5_1,GE,1),
 docplex.mp.LinearConstraint[](x_1_2+x_2_2+x_3_2+x_4_2+x_5_2,GE,1)]

In [29]:
m.print_information()

Model: Découpage éléctoral
 - number of variables: 26
   - binary=26, integer=0, continuous=0
 - number of constraints: 12
   - linear=12
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [30]:
m.add_constraints(sum(nbVi[i-1]*x[i,j] for i in range(1, n+1)) <= ((1.2)*sum(nbVi)/k) for j in range(1,k+1))




[docplex.mp.LinearConstraint[](5x_1_1+6x_2_1+5x_3_1+6x_4_1+7x_5_1,LE,17.4),
 docplex.mp.LinearConstraint[](5x_1_2+6x_2_2+5x_3_2+6x_4_2+7x_5_2,LE,17.4)]

In [31]:
m.add_constraints(sum(nbVi[i-1]*x[i,j] for i in range(1, n+1)) >= ((0.8)*sum(nbVi)/k) for j in range(1,k+1))


[docplex.mp.LinearConstraint[](5x_1_1+6x_2_1+5x_3_1+6x_4_1+7x_5_1,GE,11.600000000000001),
 docplex.mp.LinearConstraint[](5x_1_2+6x_2_2+5x_3_2+6x_4_2+7x_5_2,GE,11.600000000000001)]

In [32]:
m.add_constraints(sum(nbVi[i-1]*y[i,j] for i in range(1, n+1)) <= sum(pVi[i-1]*nbVi[i-1]*x[i,j]+(0.5)*nbVi[i-1]*x[i,j] for i in range(1, n+1)) for j in range(1,k+1))


[docplex.mp.LinearConstraint[](5y_1_1+6y_2_1+5y_3_1+6y_4_1+7y_5_1,LE,4.500x_1_1+6.600x_2_1+6x_3_1+4.200x_4_1+9.800x_5_1),
 docplex.mp.LinearConstraint[](5y_1_2+6y_2_2+5y_3_2+6y_4_2+7y_5_2,LE,4.500x_1_2+6.600x_2_2+6x_3_2+4.200x_4_2+9.800x_5_2)]

In [33]:
for i in range(1,n+1):
    for j in range(1,k+1):
        m.add_constraint(y[i,j] <= (0.5)*(z[j-1]+x[i,j]))


In [34]:
m.print_information()

Model: Découpage éléctoral
 - number of variables: 26
   - binary=26, integer=0, continuous=0
 - number of constraints: 28
   - linear=28
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [35]:
for i in range(1,n+1):
    for j in range(1,k+1):
        m.add_constraint(y[i,j] >= z[j-1]+x[i,j]-1)

In [36]:
m.print_information()

Model: Découpage éléctoral
 - number of variables: 26
   - binary=26, integer=0, continuous=0
 - number of constraints: 38
   - linear=38
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Goals

In [41]:
#Fonction que l'on cherche à optimiser 

m.minimize(sum(z))


# Solution

In [42]:
sol = m.solve()

In [43]:
sol.display()


solution for: Découpage éléctoral
objective: 0
x_1_2 = 1
x_2_2 = 1
x_3_2 = 1
x_4_1 = 1
x_5_1 = 1


In [45]:
m.maximize(sum(z))


In [46]:
solmax = m.solve()

In [47]:
solmax.display()


solution for: Découpage éléctoral
objective: 2
x_1_1 = 1
x_2_1 = 1
x_3_1 = 1
x_4_2 = 1
x_5_2 = 1
y_1_1 = 1
y_2_1 = 1
y_3_1 = 1
y_4_2 = 1
y_5_2 = 1
z_1 = 1
z_2 = 1


In [53]:
for j in range(1,k+1):
    print(z[j-1].solution_value)
    for i in range(1,n+1):
        print("x","_",i,"_", j, " = ", x[i,j].solution_value)
        y[i,j].solution_value

1.0
x _ 1 _ 1  =  1.0
x _ 2 _ 1  =  1.0
x _ 3 _ 1  =  1.0
x _ 4 _ 1  =  0
x _ 5 _ 1  =  0
1.0
x _ 1 _ 2  =  0
x _ 2 _ 2  =  0
x _ 3 _ 2  =  0
x _ 4 _ 2  =  1.0
x _ 5 _ 2  =  1.0
